In [82]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [83]:
# from huggingface_hub import notebook_login
# notebook_login()

In [84]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
label2id = {'Corrective':0,'Adaptive':1,'Perfective':2}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
# train = train.rename(columns={'3_labels':'label','comment':'text'})
train = train.replace({"label": label2id})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
# test = test.rename(columns={'3_labels':'label','comment':'text'})
test = test.replace({"label": label2id})
test.fillna(0, inplace=True)
print(len(test))



538


In [85]:
# from datasets import Dataset, load_metric

In [86]:
# train_dataset = Dataset.from_pandas(train)
# test_dataset = Dataset.from_pandas(test)

In [87]:
len(train)

1255

In [88]:
train['label'].value_counts()

0    422
2    420
1    413
Name: label, dtype: int64

In [89]:
test['label'].value_counts()

0    181
2    180
1    177
Name: label, dtype: int64

In [90]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-mpnet-base-v2')

In [91]:
train['embeddings'] = train['text'].apply(model.encode)
test['embeddings'] = test['text'].apply(model.encode)

In [92]:
test

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder,embeddings
236,d1bdfa2c80fa011aef572a598e7a1e0671f70e25,intellij-community,cleanup--,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0.028679524, 0.15876327, -0.01946884, -0.0878..."
1430,d008929c8439edf66bd9bb428201217304eacfea,Vala,libxml-2.0: Fix instance position in several m...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0.08965536, 0.12878785, 0.014861485, 0.099632..."
1039,f5a3477fe1bd136a3f9f7e2048299ad24bd47ded,Valadoc,gtkdoc-importer: Add support for computeroutput\n,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.17089736, 0.069848426, -0.022008818, 0.075..."
535,7be3d247f90c23bbb11bc735276d51008243a7f6,kotlin,fix tests after recent refactoring--,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.10820367, -0.107596554, -0.021817751, -0.0..."
1135,a449a6993aa2850d828e87676210224b6f24d35b,tapiji,updated licenses\n,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.15400934, 0.11351352, -0.05374658, -0.1141..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,6945d04785d7322df45b5c71053ba18431f606cd,orientdb,Further work or traverse: - supported new- OSQ...,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.10305861, 0.14902893, -0.11710429, -0.0521..."
47,d43b3d1f76bc13cd4c2bf8cb4f951f751aea86e6,ReactiveX-RxJava,Fix autoConnect calling onStart twice.--,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.02289819, -0.11064344, -0.08303074, 0.0400..."
424,8c475876dda2507977fd7282c37462136400daf2,drools,-Added fixes for waltz to run waltz50--git-svn...,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.106913984, 0.2002657, 0.00640438, 0.034036..."
660,ebfb482c3ee955f24be4d2802c1e06d6f0aa0d1c,restlet-framework-java,Removed debug traces.--,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.15994847, 0.010602998, 0.08137682, 0.10142..."


In [93]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [94]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [95]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression F1-score:",metrics.f1_score(y_test, predicted,average='weighted'))

Logistic Regression Accuracy: 0.7100371747211895
Logistic Regression Precision: 0.7100809357028974
Logistic Regression Recall: 0.7100371747211895
Logistic Regression Recall: 0.7098174135198438
